In [1]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import wandb

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader

from torch.utils import data


import copy
import os
import random
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import functools
from tqdm import tqdm
from datetime import datetime
import numpy as np
from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator,VQFullMotionDataset
from einops import rearrange, reduce, pack, unpack
import librosa

In [4]:
from core.datasets.vqa_motion_dataset import MotionCollatorConditional, TransMotionDatasetConditional,VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator,VQFullMotionDataset
from sklearn.metrics.pairwise import cosine_similarity

In [506]:
for batch in aist_loader:
    break

In [507]:
batch["motion"].shape

torch.Size([1, 147, 263])

In [5]:
from core.models.eval_modules import AISTEncoderBiGRUCo

from configs.config import cfg, get_cfg_defaults
from core.models.vqvae import VQMotionModel
from core.models.motion_regressor import MotionRegressorModel


cfg_e = get_cfg_defaults()
cfg_e.merge_from_file("/srv/scratch/sanisetty3/music_motion/motion_vqvae/checkpoints/extractors/small/aist_extractor.yaml")


motion_extractor = AISTEncoderBiGRUCo(cfg_e.extractor.motion_input_size,cfg_e.extractor.hidden_size,cfg_e.extractor.output_size)
music_extractor =  AISTEncoderBiGRUCo(cfg_e.extractor.music_input_size,cfg_e.extractor.hidden_size,cfg_e.extractor.output_size)


In [59]:
chk = torch.load("/srv/scratch/sanisetty3/music_motion/motion_vqvae/checkpoints/extractors/small/checkpoints/extractors.120000.pt")
print(chk["steps"])

tensor([120000.], device='cuda:0')


In [60]:
motion_extractor.load_state_dict(chk["motion_extractor"])
music_extractor.load_state_dict(chk["music_extractor"])
motion_extractor.eval()
music_extractor.eval()

AISTEncoderBiGRUCo(
  (input_emb): Linear(in_features=128, out_features=256, bias=True)
  (gru): GRU(256, 256, batch_first=True, bidirectional=True)
  (output_net): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): Linear(in_features=256, out_features=64, bias=True)
  )
)

100%|██████████| 1910/1910 [00:03<00:00, 534.79it/s]

Total number of motions 1910


In [9]:
gt_musics_list = glob("/srv/scratch/sanisetty3/music_motion/AIST/music/*.npy")

In [10]:
(gt_musics_list[0])

'/srv/scratch/sanisetty3/music_motion/AIST/music/mWA0.npy'

In [11]:
music_encodings = []
music_encodings_name = []
for i in (gt_musics_list):
    musc = torch.Tensor(np.load(i))
    ec = music_extractor(musc[None,...],torch.Tensor([musc.shape[0]]))
    music_encodings.append(ec)
    music_encodings_name.append(os.path.basename(i).split(".")[0])

In [12]:
music_encodings_np = torch.cat(music_encodings, dim=0).detach().cpu().numpy()

In [13]:
music_encodings_np.shape

(60, 64)

In [ ]:
from core.datasets.evaluator_dataset import EvaluatorMotionCollator, EvaluatorVarLenMotionDataset,EvaluatorMotionDataset
# aist_ds = EvaluatorVarLenMotionDataset(split = "test" ,data_root = "/srv/scratch/sanisetty3/music_motion/AIST" ,num_stages = 6 ,min_length_seconds=10, max_length_seconds=40)
aist_ds = EvaluatorMotionDataset(split = "train" ,data_root = "/srv/scratch/sanisetty3/music_motion/AIST" ,window_size = -1 )

collate_fn = EvaluatorMotionCollator()

aist_loader = DATALoader(aist_ds,1,collate_fn=collate_fn)

In [61]:
em_ = []
ec_ = []
names = []

for batch in tqdm(aist_loader):
#     if  int(batch["motion_lengths"][0]) < 600:
#         continue
#     if len(em_) > 40:
#         break

    music_name = batch["names"][0].split("_")[-2]
    

    if music_name not in names:
        names.append(music_name)

        em = motion_extractor(batch["motion"], batch["motion_lengths"])
        ec = music_extractor(batch["condition"],batch["motion_lengths"])
        em_.append(em)
        ec_.append(ec)


100%|██████████| 1910/1910 [00:11<00:00, 168.27it/s]


In [92]:
em_[0].shape

torch.Size([1, 64])

In [62]:
em_np = torch.cat(em_, dim=0).detach().cpu().numpy()
ec_np = torch.cat(ec_, dim=0).detach().cpu().numpy()

In [63]:
len(names)

52

In [64]:
F.cosine_similarity(torch.Tensor(em_[0]),torch.Tensor(ec_[2]) , dim=1)

tensor([0.1562], grad_fn=<SumBackward1>)

In [65]:
F.cosine_similarity(torch.Tensor(em_np),torch.Tensor(ec_np) , dim=1)

tensor([ 0.9913,  0.5768,  0.1613, -0.0533,  0.4460,  0.2672,  0.4311,  0.9829,
         0.9856,  0.9898, -0.0078,  0.9932, -0.0476,  0.2012,  0.9919,  0.5833,
         0.9859,  0.9884,  0.3937,  0.9925,  0.1603, -0.0210,  0.2217,  0.9904,
         0.6562,  0.6452,  0.3598,  0.9829,  0.3401,  0.1761,  0.1444,  0.9941,
         0.6559,  0.4310, -0.0039,  0.4904, -0.1530,  0.9917,  0.9874,  0.9887,
        -0.0311,  0.2942, -0.0044,  0.1604, -0.0897,  0.9888,  0.9911,  0.4894,
         0.2795,  0.7688,  0.9928,  0.1289])

In [67]:
print(cosine_similarity(em_np , ec_np)[4:8,4:8])

[[ 0.44604453  0.08645589 -0.20496403  0.0420238 ]
 [ 0.08076429  0.26715535 -0.07636566 -0.15384707]
 [-0.05276822 -0.0871304   0.431055    0.06496669]
 [ 0.02543228 -0.03414407 -0.07645725  0.9829299 ]]


In [298]:
indx = 2
print(music_encodings_name[indx], [names.index(music_encodings_name[indx])])
for i in range(60):
    print("music: ", music_encodings_name[i] , "motion :", names[names.index(music_encodings_name[indx])] , (F.cosine_similarity(music_encodings[i],em_[names.index(music_encodings_name[indx])] , dim=1)).detach().cpu().numpy()[0])
    

mJB0 [25]
music:  mWA0 motion : mJB0 -0.08787343
music:  mHO0 motion : mJB0 0.032452673
music:  mJB0 motion : mJB0 0.36589286
music:  mMH0 motion : mJB0 0.27313977
music:  mPO0 motion : mJB0 -0.00822898
music:  mJS0 motion : mJB0 0.15264952
music:  mBR3 motion : mJB0 0.057543725
music:  mBR4 motion : mJB0 -0.14038682
music:  mLH0 motion : mJB0 0.12863988
music:  mLO2 motion : mJB0 -0.10100515
music:  mKR1 motion : mJB0 0.0504803
music:  mLO5 motion : mJB0 0.06902988
music:  mMH1 motion : mJB0 0.15503535
music:  mPO1 motion : mJB0 0.08475751
music:  mJB1 motion : mJB0 0.07236068
music:  mHO1 motion : mJB0 0.20598385
music:  mWA1 motion : mJB0 0.0021195784
music:  mKR0 motion : mJB0 0.11459133
music:  mLO4 motion : mJB0 0.048606966
music:  mLO3 motion : mJB0 -0.05313343
music:  mLH1 motion : mJB0 -0.0067003807
music:  mBR5 motion : mJB0 0.023721173
music:  mJS1 motion : mJB0 0.15082487
music:  mBR2 motion : mJB0 0.04827468
music:  mKR4 motion : mJB0 0.064786606
music:  mLO0 motion : mJB0

## Style

In [69]:
from utils.eval_music import evaluate_music_motion_generative_style, evaluate_music_motion_generative_style2

In [70]:
import clip
clip_model, clip_preprocess = clip.load("ViT-B/32", device=torch.device('cuda'), jit=False)  # Must set jit=False for training
clip_model.eval()
for p in clip_model.parameters():
    p.requires_grad = False


In [71]:
encodec_style = "/srv/scratch/sanisetty3/music_motion/motion_vqvae/checkpoints/generator/var_len/trans_768_768_albi_aist_style/var_len_768_768_aist_style.yaml"


In [75]:
aist_ds = VQFullMotionDataset("aist", split = "train" , data_root = "/srv/scratch/sanisetty3/music_motion/AIST" , window_size = -1)
aist_loader = DATALoader(aist_ds,1,collate_fn=None)

100%|██████████| 1910/1910 [00:01<00:00, 1009.06it/s]

Total number of motions 1910


In [72]:
from configs.config import cfg, get_cfg_defaults
from core.models.motion_regressor import MotionRegressorModel


cfg_trans = get_cfg_defaults()
cfg_trans.merge_from_file(encodec_style)



trans_model = MotionRegressorModel(args = cfg_trans.motion_trans,pad_value=1025 ).eval()
pkg_trans = torch.load(f"{os.path.dirname(encodec_style)}/trans_motion.pt", map_location = 'cpu')
print(pkg_trans["steps"])
trans_model.load_state_dict(pkg_trans["model"])
trans_model =trans_model.cuda()


tensor([95000.])


In [199]:

genre_dict = {
"mBR" : "Break",
"mPO" : "Pop",
"mLO" : "Lock",
"mMH" : "Middle Hip-hop",
"mLH" : "LA style Hip-hop",
"mHO" : "House",    
"mWA" : "Waack",
"mKR" : "Krump",
"mJS" : "Street Jazz",
"mJB" : "Ballet Jazz",
}
audio_feature_dir = "/srv/scratch/sanisetty3/music_motion/AIST/audio_features"
audio_encoding_dir = "/srv/scratch/sanisetty3/music_motion/AIST/music"


In [195]:

def getscore(ec,em):
    return (F.cosine_similarity(ec,em , dim=1)).detach().cpu().numpy()[0]

def get_frequent(similarity_list):
    trim = []
    for i in similarity_list[:5]:
        trim.append(i[:3])
    return max(set(trim), key = trim.count)

In [214]:
for i,aist_batch in enumerate(tqdm(aist_loader)):
    break

  0%|          | 0/1910 [00:00<?, ?it/s]


In [215]:
motion_name = aist_batch["names"][0]

music_name = motion_name.split('_')[-2]
print(music_name)

mPO4


In [216]:
og_added_style = []
pred_aded_style = []
for style_ in genre_dict.values():

    style = style_
    mot_len = aist_batch["motion_lengths"][0]
    motion_name = aist_batch["names"][0]

    music_name = motion_name.split('_')[-2]
    music_encoding=  np.load(os.path.join(audio_encoding_dir , music_name + ".npy"))

    genre = (genre_dict.get(music_name[:3])) if style is None else style

    text = clip.tokenize([genre], truncate=True).cuda()
    style_embeddings = clip_model.encode_text(text).cpu().float().reshape(-1) if clip_model is not None else None

    print(motion_name)
    print(music_name)
    print("added: ", genre)

    seq_len = 200
    gen_motion_indices = torch.randint(0 , 1024 , (1,1))

    gen_motion_indices = trans_model.generate(start_tokens =gen_motion_indices.cuda(),\
                                    seq_len=seq_len , \
                                    context = torch.Tensor(music_encoding)[None,...].cuda(), \
                                    context_mask=torch.ones((1 ,music_encoding.shape[0]) , dtype = torch.bool).cuda(), \
                                    style_context = torch.Tensor(style_embeddings.reshape(-1))[None,...].cuda(),

                                    )

    out_motion = torch.zeros((aist_batch["motion"].shape[0] ,gen_motion_indices.shape[-1] , aist_batch["motion"].shape[-1]))
    for i in range(0 , seq_len, 200):
        quant , out_motion_= vqvae_model.decode(gen_motion_indices[:,i:i+200])
        out_motion[:,i:i+200] = out_motion_

    out_motion = out_motion[:,1:]
    
    em = motion_extractor(out_motion, torch.Tensor([seq_len]))
    similarity_name_list  =sorted(music_encodings_name, key= lambda x: getscore(music_encodings[music_encodings_name.index(x)] , em ), reverse = True)
    try: 
        added_music =  list(genre_dict.keys())[list(genre_dict.values()).index(style)]
    except:
        added_music = None
        
        
    pred_add_music = get_frequent(similarity_name_list)
    print( f"og {music_name} + add {added_music}" , f"music most similar to: {pred_add_music}")
    
    og_added_style.append(added_music)
    pred_aded_style.append(pred_add_music)
    
cnt = 0
for i in range(len(og_added_style)):
    cnt+=og_added_style[i]==pred_aded_style[i]
cnt/len(og_added_style)
    

  2%|▏         | 4/200 [00:00<00:06, 30.54it/s]

M_gPO_sBM_cAll_d11_mPO4_ch06
mPO4
added:  Break


  2%|▏         | 4/200 [00:00<00:05, 33.85it/s]

og mPO4 + add mBR music most similar to: mJB
M_gPO_sBM_cAll_d11_mPO4_ch06
mPO4
added:  Pop


  2%|▏         | 4/200 [00:00<00:06, 32.25it/s]

...

  2%|▏         | 4/200 [00:00<00:05, 32.87it/s]

og mPO4 + add mJS music most similar to: mJB
M_gPO_sBM_cAll_d11_mPO4_ch06
mPO4
added:  Ballet Jazz


100%|██████████| 200/200 [00:07<00:00, 27.04it/s]


og mPO4 + add mJB music most similar to: mJB


0.1

0.2

og mJS2 + add mLH music most similar to: mLH ['mJS5', 'mLH1', 'mLH2', 'mMH5', 'mLH4']


In [131]:
for i in indxs:
    print("music: ", music_encodings_name[i] , "motion :", music_name, (F.cosine_similarity(music_encodings[i],em , dim=1)).detach().cpu().numpy()[0])
    

og mBR0 + add mBR motion : mLH5 0.22952467
og mBR1 + add mBR motion : mLH5 0.31058973
og mBR2 + add mBR motion : mLH5 0.36543846
og mBR3 + add mBR motion : mLH5 0.34459582
og mBR4 + add mBR motion : mLH5 0.17643668
og mBR5 + add mBR motion : mLH5 0.3106068
og mHO0 + add mBR motion : mLH5 0.24023971
og mHO1 + add mBR motion : mLH5 0.30225283
og mHO2 + add mBR motion : mLH5 -0.0542868
og mHO3 + add mBR motion : mLH5 0.03304533
og mHO4 + add mBR motion : mLH5 0.10309335
og mHO5 + add mBR motion : mLH5 0.1036674
og mJB0 + add mBR motion : mLH5 0.17873418
og mJB1 + add mBR motion : mLH5 0.089289896
og mJB2 + add mBR motion : mLH5 0.1746877
og mJB3 + add mBR motion : mLH5 0.3168804
og mJB4 + add mBR motion : mLH5 0.116581395
og mJB5 + add mBR motion : mLH5 0.105608374
og mJS0 + add mBR motion : mLH5 -0.0076647885
og mJS1 + add mBR motion : mLH5 0.4384472
og mJS2 + add mBR motion : mLH5 0.12914537
og mJS3 + add mBR motion : mLH5 0.35537
og mJS4 + add mBR motion : mLH5 0.050492648
og mJS5 + ad

## Evaluators

In [301]:
encodec = "/srv/scratch/sanisetty3/music_motion/motion_vqvae/checkpoints/generator/var_len/trans_768_768_albi_aist/var_len_768_768_aist.yaml"
encodec_sine = "/srv/scratch/sanisetty3/music_motion/motion_vqvae/checkpoints/generator/var_len/trans_768_768_sine_aist/var_len_768_768_sine_aist.yaml"
librosa = "/srv/scratch/sanisetty3/music_motion/motion_vqvae/checkpoints/generator/var_len/trans_768_768_albi_aist_35/var_len_768_768_aist_35.yaml"
encodec_prob50 = "/srv/scratch/sanisetty3/music_motion/motion_vqvae/checkpoints/generator/var_len/trans_768_768_albi_aist_mask_prob50/trans_768_768_albi_aist_mask_prob50.yaml"


In [302]:
from configs.config import cfg, get_cfg_defaults
from core.models.motion_regressor import MotionRegressorModel


trans_option = "encodec"

cfg_trans = get_cfg_defaults()
cfg_trans.merge_from_file(encodec)


trans_model = MotionRegressorModel(args = cfg_trans.motion_trans,pad_value=1025 ).eval()
pkg_trans = torch.load(f"{os.path.dirname(encodec)}/trans_motion_best_fid.pt", map_location = 'cpu')
print(pkg_trans["steps"])
trans_model.load_state_dict(pkg_trans["model"])
trans_model =trans_model.cuda()


tensor([210000.])


In [87]:
from configs.config import cfg, get_cfg_defaults
from core.models.vqvae import VQMotionModel
from core.models.motion_regressor import MotionRegressorModel


cfg_vq = get_cfg_defaults()
cfg_vq.merge_from_file("/srv/scratch/sanisetty3/music_motion/motion_vqvae/configs/var_len_768_768_aist_vq.yaml")




In [88]:
vqvae_model = VQMotionModel(cfg_vq.vqvae).eval()
pkg = torch.load(f"/srv/scratch/sanisetty3/music_motion/motion_vqvae/checkpoints/var_len/vq_768_768_mix/vqvae_motion_best_fid.pt", map_location = 'cpu')
print(pkg["steps"])
vqvae_model.load_state_dict(pkg["model"])
vqvae_model =vqvae_model.cuda()


tensor([295000.])


In [110]:
from core.models.evaluator_wrapper import AISTEvaluatorModelWrapper
from utils.eval_music import evaluate_music_motion_generative_extractors
from utils.eval_trans import calculate_R_precision,calculate_activation_statistics,calculate_diversity,calculate_frechet_distance


In [116]:
from configs.config import cfg, get_cfg_defaults

cfg_eval = get_cfg_defaults()
cfg_eval.merge_from_file("/srv/scratch/sanisetty3/music_motion/motion_vqvae/checkpoints/extractors/big/aist_extractor.yaml")

In [621]:
from core.datasets.evaluator_dataset import EvaluatorMotionCollator, EvaluatorVarLenMotionDataset, EvaluatorMotionDataset
aist_ds = EvaluatorMotionDataset(split = "test" ,data_root = "/srv/scratch/sanisetty3/music_motion/AIST" ,window_size = -1 )
collate_fn = EvaluatorMotionCollator()

aist_loader = DATALoader(aist_ds,20,collate_fn=collate_fn)

100%|██████████| 40/40 [00:00<00:00, 784.15it/s]

Total number of motions 40


In [515]:
aist_evaluator = AISTEvaluatorModelWrapper(cfg_eval)

loading from:  /srv/scratch/sanisetty3/music_motion/motion_vqvae/checkpoints/extractors/big/extractors.pt steps:  tensor([95000.], device='cuda:0')


In [517]:
fids = []
div_R = []
div_p = []

for i in range(20):
    fid, diversity_real, diversity = evaluate_music_motion_generative_extractors(aist_loader , vqvae_model= vqvae_model ,net = trans_model, eval_wrapper = aist_evaluator, seq_len = 600)
    fids.append(fid)
    div_R.append(diversity_real)
    div_p.apppend(diversity)

In [622]:


seq_len = 600
motion_annotation_list = []
motion_pred_list = []

music_annotation_list = []
music_pred_list = []

R_precision_real = 0
R_precision = 0

nb_sample = 0

audio_dir = "/srv/scratch/sanisetty3/music_motion/AIST/music"

matching_score_real = 0
matching_score_pred = 0
motion_extractor = motion_extractor.cuda()
music_extractor = music_extractor.cuda()

for j,aist_batch in enumerate(tqdm(aist_loader)):
    
#     if j>3:
#         break
    
    bs, seq = aist_batch["motion"].shape[0], aist_batch["motion"].shape[1]
    
    em = motion_extractor(aist_batch["motion"].cuda(), aist_batch["motion_lengths"].cuda())
    et = music_extractor(aist_batch["condition"].cuda(),aist_batch["motion_lengths"].cuda())
    
    generated_motion = torch.zeros(aist_batch["motion"].shape).cuda()
    
    for i in range(bs):
        
        mot_len = int(aist_batch["motion_lengths"][i])
        motion_name = aist_batch["names"][i]

        music_name = motion_name.split('_')[-2]
        music_encoding=  np.load(os.path.join(audio_dir , music_name + ".npy"))

        music_name = motion_name.split('_')[-2]
        gen_motion_indices = torch.randint(0 , 1024 , (1,1))

        gen_motion_indices = trans_model.generate(start_tokens =gen_motion_indices.cuda(),\
                                                    seq_len=mot_len , \
                                                    context = torch.Tensor(music_encoding)[None,...].cuda(), \
                                                    context_mask=torch.ones((1 ,music_encoding.shape[0]) , dtype = torch.bool).cuda()
                                                    )


        out_motion = torch.zeros((1, gen_motion_indices.shape[-1] , aist_batch["motion"].shape[-1]))
        for j in range(0 , mot_len, 200):
            quant , out_motion_= vqvae_model.decode(gen_motion_indices[:,j:j+200])
            out_motion[:,j:j+200] = out_motion_
            
        generated_motion[i:i+1,:mot_len,:] = out_motion[:,1:,:]
        
#         print(out_motion.shape)
        

    em_pred = motion_extractor(generated_motion.cuda(), aist_batch["motion_lengths"].cuda())
    et_pred = music_extractor(aist_batch["condition"].cuda(),aist_batch["motion_lengths"].cuda())
    


    motion_pred_list.append(em_pred)
    motion_annotation_list.append(em)

    music_pred_list.append(et_pred)
    music_annotation_list.append(et)
    
    
    temp_R, temp_match = calculate_R_precision(et.detach().cpu().numpy(), em.detach().cpu().numpy(), top_k=3, sum_all=True)
    R_precision_real += temp_R
    matching_score_real += temp_match

    temp_R_pred, temp_match_pred = calculate_R_precision(et_pred.detach().cpu().numpy(), em_pred.detach().cpu().numpy(), top_k=3, sum_all=True)
    R_precision += temp_R_pred
    matching_score_pred += temp_match_pred
    
    



    nb_sample += bs
    
motion_annotation_np = torch.cat(motion_annotation_list, dim=0).detach().cpu().numpy()
motion_pred_np = torch.cat(motion_pred_list, dim=0).detach().cpu().numpy()

music_annotation_np = torch.cat(music_annotation_list, dim=0).detach().cpu().numpy()
music_pred_np = torch.cat(music_pred_list, dim=0).detach().cpu().numpy()

gt_mu, gt_cov  = calculate_activation_statistics(motion_annotation_np)
mu, cov= calculate_activation_statistics(motion_pred_np)
fid = calculate_frechet_distance(gt_mu, gt_cov, mu, cov)
print("fid ", fid)

diversity_real = calculate_diversity(motion_annotation_np, 100 if nb_sample > 100 else 10)
diversity = calculate_diversity(motion_pred_np, 100 if nb_sample > 100 else 10)
print(diversity_real , diversity)




100%|██████████| 2/2 [02:22<00:00, 71.09s/it]

fid  0.6284469415386564
1.1949672 1.1772099


In [629]:
  
motion_annotation_np = torch.cat(motion_annotation_list, dim=0).detach().cpu().numpy()
motion_pred_np = torch.cat(motion_pred_list, dim=0).detach().cpu().numpy()

music_annotation_np = torch.cat(music_annotation_list, dim=0).detach().cpu().numpy()
music_pred_np = torch.cat(music_pred_list, dim=0).detach().cpu().numpy()

gt_mu, gt_cov  = calculate_activation_statistics(motion_annotation_np)
mu, cov= calculate_activation_statistics(motion_pred_np)
fid = calculate_frechet_distance(gt_mu, gt_cov, mu, cov)
print("fid ", fid)

diversity_real = calculate_diversity(motion_annotation_np, 100 if nb_sample > 100 else 30)
diversity = calculate_diversity(motion_pred_np, 100 if nb_sample > 100 else 30)
print(diversity_real , diversity)

fid  0.6284469415386564
1.1452008 1.1187509


In [624]:
motion_annotation_np.shape

(40, 64)

In [625]:
print(R_precision_real/motion_annotation_np.shape[0])
print(R_precision/motion_annotation_np.shape[0])

print(matching_score_real/motion_annotation_np.shape[0])
print(matching_score_pred/motion_annotation_np.shape[0])

[0.075 0.1   0.1  ]
[0.05  0.125 0.175]
2.016997146606445
2.1148876190185546


In [628]:
ln = music_annotation_np.shape[0]
temp_R, temp_match = calculate_R_precision(music_annotation_np, motion_annotation_np, top_k=3, sum_all=True)
print(temp_R/ln, temp_match/ln)
temp_R_pred, temp_match_pred = calculate_R_precision(music_annotation_np,motion_pred_np, top_k=3, sum_all=True)
print(temp_R_pred/ln, temp_match_pred/ln )

[0.025 0.05  0.1  ] 2.016996955871582
[0.    0.05  0.075] 2.1148876190185546


## Eval on EDGE

In [220]:
import pickle
from scipy.spatial.transform import Rotation as R


In [221]:
edge_motions = glob("/srv/scratch/sanisetty3/music_motion/EDGE/eval/hml/new_joint_vecs/*.npy")


In [219]:
edge_motions = glob("/srv/scratch/sanisetty3/music_motion/EDGE/eval/motions/*.pkl")
aist_motions = glob("/srv/scratch/sanisetty3/clean/mint/evals/eval60/g*.npy")

In [553]:
list(set(names))

['mKR2',
 'mWA0',
 'mPO1',
 'mHO5',
 'mLO2',
 'mJB5',
 'mLH4',
 'mBR0',
 'mMH3',
 'mJS3']

In [573]:
set(names) & set([i.split(".")[0][-4:] for i in edge_motions])

{'mBR0', 'mHO5', 'mJS3', 'mKR2', 'mLH4', 'mLO2', 'mMH3', 'mPO1', 'mWA0'}

In [612]:
hml_msc = ['mBR0', 'mHO5', 'mJS3', 'mKR2', 'mLH4', 'mLO2', 'mMH3', 'mPO1', 'mWA0']

In [614]:
torch.Tensor(np.load(f"/srv/scratch/sanisetty3/music_motion/EDGE/eval/hml/new_joint_vecs/{music_name}.npy")).shape

torch.Size([899, 263])

In [616]:
from core.datasets.evaluator_dataset import EvaluatorMotionCollator, EvaluatorVarLenMotionDataset, EvaluatorMotionDataset
aist_ds = EvaluatorMotionDataset(split = "test" ,data_root = "/srv/scratch/sanisetty3/music_motion/AIST" ,window_size = -1 )
collate_fn = EvaluatorMotionCollator()

aist_loader = DATALoader(aist_ds,1,collate_fn=collate_fn)

100%|██████████| 40/40 [00:00<00:00, 809.99it/s]

Total number of motions 40


In [618]:


seq_len = 600
motion_annotation_list = []
motion_pred_list = []

music_annotation_list = []
music_pred_list = []

R_precision_real = 0
R_precision = 0

nb_sample = 0

audio_dir = "/srv/scratch/sanisetty3/music_motion/AIST/music"

matching_score_real = 0
matching_score_pred = 0
motion_extractor = motion_extractor.cuda()
music_extractor = music_extractor.cuda()

for j,aist_batch in enumerate(tqdm(aist_loader)):
    
#     if j>3:
#         break
    
    bs, seq = aist_batch["motion"].shape[0], aist_batch["motion"].shape[1]
    
    
    mot_len = int(aist_batch["motion_lengths"][0])

    em = motion_extractor(aist_batch["motion"].cuda(), aist_batch["motion_lengths"].cuda())
    et = music_extractor(aist_batch["condition"].cuda(),aist_batch["motion_lengths"].cuda())
    
    motion_name = aist_batch["names"][0]
    
    music_name = motion_name.split('_')[-2]
    print(music_name)
    music_encoding=  np.load(os.path.join(audio_dir , music_name + ".npy"))
    

    try:
        generated_motion = torch.Tensor(np.load(f"/srv/scratch/sanisetty3/music_motion/EDGE/eval/hml/new_joint_vecs/{music_name}.npy"))[:mot_len,:][None,...]
    except:
        continue
                                   

    em_pred = motion_extractor(generated_motion.cuda(), aist_batch["motion_lengths"].cuda())
    et_pred = music_extractor(aist_batch["condition"].cuda(),aist_batch["motion_lengths"].cuda())
    


    motion_pred_list.append(em_pred)
    motion_annotation_list.append(em)

    music_pred_list.append(et_pred)
    music_annotation_list.append(et)
    




    nb_sample += bs
    
motion_annotation_np = torch.cat(motion_annotation_list, dim=0).detach().cpu().numpy()
motion_pred_np = torch.cat(motion_pred_list, dim=0).detach().cpu().numpy()

music_annotation_np = torch.cat(music_annotation_list, dim=0).detach().cpu().numpy()
music_pred_np = torch.cat(music_pred_list, dim=0).detach().cpu().numpy()

gt_mu, gt_cov  = calculate_activation_statistics(motion_annotation_np)
mu, cov= calculate_activation_statistics(motion_pred_np)
fid = calculate_frechet_distance(gt_mu, gt_cov, mu, cov)
print("fid ", fid)

diversity_real = calculate_diversity(motion_annotation_np, 100 if nb_sample > 100 else 10)
diversity = calculate_diversity(motion_pred_np, 100 if nb_sample > 100 else 10)
print(diversity_real , diversity)


temp_R, temp_match = calculate_R_precision(music_annotation_np, motion_annotation_np, top_k=3, sum_all=True)
print(temp_R, temp_match)
temp_R_pred, temp_match_pred = calculate_R_precision(music_pred_np,motion_pred_np, top_k=3, sum_all=True)
print(temp_R_pred, temp_match_pred )

 15%|█▌        | 6/40 [00:00<00:00, 59.19it/s]

mKR2
mPO1
mLH4
mWA0
mJS3
mMH3
mKR2
mHO5
mBR0
mMH3
mHO5
mMH3
mJS3


 70%|███████   | 28/40 [00:00<00:00, 65.08it/s]

mLH4
mJB5
mKR2
mWA0
mLH4
mHO5
mJB5
mJB5
mJB5
mPO1
mLH4
mLO2
mMH3
mLO2
mJS3


 85%|████████▌ | 34/40 [00:00<00:00, 58.31it/s]

mPO1
mBR0
mLO2
mPO1
mWA0
mKR2
mHO5
mBR0
mJS3
mLO2
mBR0


100%|██████████| 40/40 [00:00<00:00, 60.82it/s]

mWA0
fid  1.6043953599870409
1.1438 0.49290603
[1 2 4] 71.93391
[1 2 3] 82.030975


In [619]:
motion_annotation_np.shape

(36, 64)

In [222]:
import librosa

In [223]:
from smplx import SMPL

# set smpl
smpl = SMPL(model_path="/srv/share/datasets/AIST/smpl_models", gender='MALE', batch_size=1)

In [224]:
from aist_plusplus.loader import AISTDataset


In [225]:
edge_motions = glob("/srv/scratch/sanisetty3/music_motion/EDGE/eval/motions/*.pkl")


In [226]:
anno_dir = "/srv/share/datasets/AIST/aist_plusplus_final/"
audio_dir = "/srv/share/datasets/AIST/aist_plusplus_media/audio/wav/"
audio_cache_dir = "/srv/scratch/sanisetty3/clean/mint/data/audio_cache"


In [227]:
seq_names = []
seq_names += np.loadtxt(
            os.path.join("/srv/scratch/sanisetty3/mint_pyt/mint/third_party/aist_plusplus_final/splits/crossmodal_test.txt"), dtype=str
        ).tolist()

ignore_list = np.loadtxt(
    os.path.join("/srv/scratch/sanisetty3/mint_pyt/mint/third_party/aist_plusplus_final/", "ignore_list.txt"), dtype=str
).tolist()
seq_names = [name for name in seq_names if name not in ignore_list]

In [228]:
len(seq_names)

20

In [721]:

def cache_audio_features(seq_names):
    FPS = 60
    HOP_LENGTH = 512
    SR = FPS * HOP_LENGTH
    EPS = 1e-6

    def _get_tempo(audio_name):
        """Get tempo (BPM) for a music by parsing music name."""
        assert len(audio_name) == 4
        if audio_name[0:3] in ['mBR', 'mPO', 'mLO', 'mMH', 'mLH', 'mWA', 'mKR', 'mJS', 'mJB']:
            return int(audio_name[3]) * 10 + 80
        elif audio_name[0:3] == 'mHO':
            return int(audio_name[3]) * 5 + 110
        else: assert False, audio_name

    audio_names = list(set([seq_name.split("_")[-2] for seq_name in seq_names]))
    print(audio_names)

    for audio_name in audio_names:
        save_path = os.path.join("/srv/scratch/sanisetty3/clean/mint/data/audio_cache", f"{audio_name}.npy")
        if os.path.exists(save_path):
            continue
        data, _ = librosa.load(os.path.join(audio_dir, f"{audio_name}.wav"), sr=SR)
        envelope = librosa.onset.onset_strength(data, sr=SR)  # (seq_len,)
        mfcc = librosa.feature.mfcc(data, sr=SR, n_mfcc=20).T  # (seq_len, 20)
        chroma = librosa.feature.chroma_cens(
            data, sr=SR, hop_length=HOP_LENGTH, n_chroma=12).T  # (seq_len, 12)

        peak_idxs = librosa.onset.onset_detect(
            onset_envelope=envelope.flatten(), sr=SR, hop_length=HOP_LENGTH)
        peak_onehot = np.zeros_like(envelope, dtype=np.float32)
        peak_onehot[peak_idxs] = 1.0  # (seq_len,)

        tempo, beat_idxs = librosa.beat.beat_track(
            onset_envelope=envelope, sr=SR, hop_length=HOP_LENGTH,
            start_bpm=_get_tempo(audio_name), tightness=100)
        beat_onehot = np.zeros_like(envelope, dtype=np.float32)
        beat_onehot[beat_idxs] = 1.0  # (seq_len,)

        audio_feature = np.concatenate([
            envelope[:, None], mfcc, chroma, peak_onehot[:, None], beat_onehot[:, None]
        ], axis=-1)
        np.save(save_path, audio_feature)



In [ ]:
cache_audio_features(seq_names)

['mKR2', 'mWA0', 'mPO1', 'mHO5', 'mLO2', 'mJB5', 'mLH4', 'mBR0', 'mMH3', 'mJS3']


In [229]:
with open(source_file, "rb") as fp:
        data = pickle.load(fp)
    
smpl_poses = data["smpl_poses"]
smpl_trans =  data["smpl_trans"]

smpl_poses_ = R.from_rotvec(smpl_poses.reshape(-1, 3)).as_matrix().reshape(smpl_poses.shape[0], -1)
motion_sequence = np.concatenate([smpl_trans, smpl_poses_], axis=-1)

NameError: name 'source_file' is not defined

In [230]:
anno_dir = "/srv/share/datasets/AIST/aist_plusplus_final/"
audio_dir = "/srv/share/datasets/AIST/aist_plusplus_media/audio/wav/"
audio_cache_dir = "/srv/scratch/sanisetty3/clean/mint/data/audio_cache"


In [231]:
def eye(n, batch_shape):
    iden = np.zeros(np.concatenate([batch_shape, [n, n]]))
    iden[..., 0, 0] = 1.0
    iden[..., 1, 1] = 1.0
    iden[..., 2, 2] = 1.0
    return iden


def get_closest_rotmat(rotmats):
    """
    Finds the rotation matrix that is closest to the inputs in terms of the Frobenius norm. For each input matrix
    it computes the SVD as R = USV' and sets R_closest = UV'. Additionally, it is made sure that det(R_closest) == 1.
    Args:
        rotmats: np array of shape (..., 3, 3).
    Returns:
        A numpy array of the same shape as the inputs.
    """
    u, s, vh = np.linalg.svd(rotmats)
    r_closest = np.matmul(u, vh)

    # if the determinant of UV' is -1, we must flip the sign of the last column of u
    det = np.linalg.det(r_closest)  # (..., )
    iden = eye(3, det.shape)
    iden[..., 2, 2] = np.sign(det)
    r_closest = np.matmul(np.matmul(u, iden), vh)
    return r_closest


def recover_to_axis_angles(motion):
    batch_size, seq_len, dim = motion.shape
    assert dim == 225
    transl = motion[:, :, 6:9]
    rotmats = get_closest_rotmat(
        np.reshape(motion[:, :, 9:], (batch_size, seq_len, 24, 3, 3))
    )
    axis_angles = R.from_matrix(
        rotmats.reshape(-1, 3, 3)
    ).as_rotvec().reshape(batch_size, seq_len, 24, 3)
    return axis_angles, transl


def recover_motion_to_keypoints(motion, smpl_model):
    smpl_poses, smpl_trans = recover_to_axis_angles(motion)
    smpl_poses = np.squeeze(smpl_poses, axis=0)  # (seq_len, 24, 3)
    smpl_trans = np.squeeze(smpl_trans, axis=0)  # (seq_len, 3)
    keypoints3d = smpl_model.forward(
        global_orient=torch.from_numpy(smpl_poses[:, 0:1]).float(),
        body_pose=torch.from_numpy(smpl_poses[:, 1:]).float(),
        transl=torch.from_numpy(smpl_trans).float(),
    ).joints.detach().numpy()[:, :24, :]   # (seq_len, 24, 3)
    return keypoints3d


def motion_peak_onehot(joints):
    """Calculate motion beats.
    Kwargs:
        joints: [nframes, njoints, 3]
    Returns:
        - peak_onhot: motion beats.
    """
    # Calculate velocity.
    velocity = np.zeros_like(joints, dtype=np.float32)
    velocity[1:] = joints[1:] - joints[:-1]
    velocity_norms = np.linalg.norm(velocity, axis=2)
    envelope = np.sum(velocity_norms, axis=1)  # (seq_len,)

    # Find local minima in velocity -- beats
    peak_idxs = scisignal.argrelextrema(envelope, np.less, axis=0, order=10) # 10 for 60FPS
    peak_onehot = np.zeros_like(envelope, dtype=bool)
    peak_onehot[peak_idxs] = 1

    # # Second-derivative of the velocity shows the energy of the beats
    # peak_energy = np.gradient(np.gradient(envelope)) # (seq_len,)
    # # optimize peaks
    # peak_onehot[peak_energy<0.001] = 0
    return peak_onehot


def alignment_score(music_beats, motion_beats, sigma=3):
    """Calculate alignment score between music and motion."""
    if motion_beats.sum() == 0:
        return 0.0
    music_beat_idxs = np.where(music_beats)[0]
    motion_beat_idxs = np.where(motion_beats)[0]
    score_all = []
    for motion_beat_idx in motion_beat_idxs:
        dists = np.abs(music_beat_idxs - motion_beat_idx).astype(np.float32)
        ind = np.argmin(dists)
        score = np.exp(- dists[ind]**2 / 2 / sigma**2)
        score_all.append(score)
    return sum(score_all) / len(score_all)



In [232]:
dataset = AISTDataset(anno_dir)
n_samples = len(seq_names)
beat_scores = []
for i, seq_name in enumerate(seq_names):
    print("processing %d / %d" % (i + 1, n_samples))
    # get real data motion beats
    smpl_poses, smpl_scaling, smpl_trans = AISTDataset.load_motion(
        dataset.motion_dir, seq_name)
    smpl_trans /= smpl_scaling
    keypoints3d = smpl.forward(
        global_orient=torch.from_numpy(smpl_poses[:, 0:1]).float(),
        body_pose=torch.from_numpy(smpl_poses[:, 1:]).float(),
        transl=torch.from_numpy(smpl_trans).float(),
    ).joints.detach().numpy()[:, :24, :]   # (seq_len, 24, 3)
    motion_beats = motion_peak_onehot(keypoints3d)
    # get real data music beats
    audio_name = seq_name.split("_")[4]
    audio_feature = np.load(os.path.join(audio_cache_dir, f"{audio_name}.npy"))
    audio_beats = audio_feature[:keypoints3d.shape[0], -1] # last dim is the music beats
    # get beat alignment scores
    beat_score = alignment_score(audio_beats, motion_beats, sigma=3)
    beat_scores.append(beat_score)
print ("\nBeat score on real data: %.3f\n" % (sum(beat_scores) / n_samples))


processing 1 / 20


NameError: name 'scisignal' is not defined

In [605]:
def process_aist(aist_vec):
    aist_ex = torch.Tensor(aist_vec)
    trans = aist_ex[:,:3]
    aist_ex_9d = aist_ex[:,3:].reshape(aist_vec.shape[0],24,3,3)
    aist_ex_9d[:,-2:] = torch.zeros((aist_vec.shape[0], 2, 3, 3))
    global_orient = aist_ex_9d[:, 0:1]
    rotations = aist_ex_9d[:, 1:]
    out_aist_og = smpl_model(body_pose=rotations, global_orient=global_orient, transl = trans)
    
            
    return  out_aist_og


In [606]:
pos = process_aist(smpl_motion)

In [608]:
pos

{'vertices': tensor([[[ 0.6551, -0.0735,  2.6354],
          [ 0.6521, -0.0824,  2.6211],
          [ 0.6559, -0.0690,  2.6174],
          ...,
          [ 0.4940, -0.0758,  2.5989],
          [ 0.4952, -0.0773,  2.6004],
          [ 0.4945, -0.0811,  2.5978]],
 
         [[ 0.6392, -0.0803,  2.6505],
          [ 0.6365, -0.0893,  2.6363],
          [ 0.6404, -0.0759,  2.6326],
          ...,
          [ 0.4787, -0.0826,  2.6107],
          [ 0.4798, -0.0841,  2.6121],
          [ 0.4792, -0.0879,  2.6096]],
 
         [[ 0.6277, -0.0911,  2.6582],
          [ 0.6250, -0.1000,  2.6440],
          [ 0.6294, -0.0868,  2.6404],
          ...,
          [ 0.4678, -0.0867,  2.6159],
          [ 0.4688, -0.0883,  2.6173],
          [ 0.4681, -0.0920,  2.6147]],
 
         ...,
 
         [[-0.0608, -0.0188,  2.6868],
          [-0.0655, -0.0282,  2.6726],
          [-0.0531, -0.0208,  2.6695],
          ...,
          [-0.1725,  0.0707,  2.6069],
          [-0.1730,  0.0690,  2.6086],
      